In [1]:
from db import Team, Athlete, Meet
import requests as r
from sqlmodel import select, Session
from db.database import SessionLocal
import json

In [2]:
api_url = 'http://127.0.0.1:8000/'

In [3]:
def get_all_teams(session: Session):
    stmt = select(Team)
    return session().exec(stmt).all()

def insert_meets_and_athletes(params):
    q = r.get(api_url + 'team/getInfo', params = params)
    meets = q.json()['schedule']
    roster = q.json()['roster']
    for meet in meets:
        response = r.post(api_url + 'meet/addMeet',data=json.dumps(meet))
        if response.status_code != 200:
            print(meet['name'], response.text)
    for athlete in roster:
        response = r.post(api_url + 'athlete/addAthlete', data=json.dumps(athlete))
        if response.status_code != 200:
            print(athlete['anet_id'], response.text)
    return

In [203]:
season = 2021
sport = 'xc'
for team in get_all_teams(SessionLocal):
    params = dict(team_id = team.anet_id, season = season, sport = sport)
    insert_meets_and_athletes(params)

17657892 {"detail":"Athlete already exists"}
17657871 {"detail":"Athlete already exists"}
17657868 {"detail":"Athlete already exists"}
17657890 {"detail":"Athlete already exists"}
15968779 {"detail":"Athlete already exists"}
17657897 {"detail":"Athlete already exists"}
17657886 {"detail":"Athlete already exists"}
17657894 {"detail":"Athlete already exists"}
15968789 {"detail":"Athlete already exists"}
17657872 {"detail":"Athlete already exists"}
17657875 {"detail":"Athlete already exists"}
19028253 {"detail":"Athlete already exists"}
15968770 {"detail":"Athlete already exists"}
18977410 {"detail":"Athlete already exists"}
17657866 {"detail":"Athlete already exists"}
17657899 {"detail":"Athlete already exists"}
18977391 {"detail":"Athlete already exists"}
18977417 {"detail":"Athlete already exists"}
18913290 {"detail":"Athlete already exists"}
17657902 {"detail":"Athlete already exists"}
18913314 {"detail":"Athlete already exists"}
18977413 {"detail":"Athlete already exists"}
17657900 {

In [4]:
from sqlalchemy import extract
def get_all_meets_by_season(session: Session, season: int):
    stmt = select(Meet).where(extract('year', Meet.date) == season)
    return session.exec(stmt).all()

In [6]:

meets = get_all_meets_by_season(SessionLocal(),2023)




In [8]:
params = dict(sport = 'xc', meet_id = 229676)
s = r.get(api_url + 'meet/getResults',params=params)
s.json()

all_results = []
for race in s.json()['races']:
    all_results += race['results']

for result in all_results:
    response = r.post(api_url + 'meet/addResult', data = json.dumps(result))
    if response.status_code == 422:
        print(response.text)

{"detail":[{"loc":["body","anet_athlete_id"],"msg":"none is not an allowed value","type":"type_error.none.not_allowed"}]}
